In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install selenium
!pip install pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
!apt-get update
!apt-get install -y wget unzip
# Download Chrome
!wget -N https://storage.googleapis.com/chrome-for-testing-public/126.0.6478.55/linux64/chrome-linux64.zip
!unzip -o chrome-linux64.zip
# Download ChromeDriver
!wget -N https://storage.googleapis.com/chrome-for-testing-public/126.0.6478.55/linux64/chromedriver-linux64.zip
!unzip -o chromedriver-linux64.zip
!chmod +x chromedriver-linux64/chromedriver
!mv -f chromedriver-linux64/chromedriver /usr/local/bin/chromedriver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,453 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,173 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/uni

In [ ]:
import os
import time
import requests
import urllib
import progressbar
from urllib.parse import quote
from google.colab import drive
import cv2
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

def _create_directories(_directory, _name):
    """
    create directory to save images
    :param _directory:
    :param _name:
    :return:
    """
    name = _name.replace(" ", "_")
    try:
        if not os.path.exists(_directory):
            os.makedirs(_directory)
            time.sleep(0.2)
            path = name
            sub_directory = os.path.join(_directory, path)
            if not os.path.exists(sub_directory):
                os.makedirs(sub_directory)
        else:
            path = name
            sub_directory = os.path.join(_directory, path)
            if not os.path.exists(sub_directory):
                os.makedirs(sub_directory)

    except OSError as e:
        if e.errno != 17:
            raise
        pass
    return

def _download_page(url):

    try:
        headers={}
        headers[
            'User-Agent']="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
        req = urllib.request.Request(url, headers=headers)
        resp = urllib.request.urlopen(req)
        respData = str(resp.read())
        return respData

    except Exception as e:
        print(e)
        exit(0)

def download_and_resize(keywords, limit, directory, resize_width, resize_height):
    """
    Download images from Google, resize them, and save them directly to Google Drive folder.
    :param keywords: Keywords to search for.
    :param limit: Limit of images to download.
    :param directory: Directory in Google Drive to save the images.
    :param resize_width: Width for resizing images.
    :param resize_height: Height for resizing images.
    """
    keyword_to_search = [str(item).strip() for item in keywords.split(',')]

    # Create directory in Google Drive if it doesn't exist
    drive_directory = os.path.join('/content/drive/MyDrive', directory)
    _create_directories(drive_directory, '')

    things = len(keyword_to_search) * limit

    bar = progressbar.ProgressBar(maxval=things, \
                                  widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

    bar.start()

    for keyword in keyword_to_search:
        url = 'https://www.google.com/search?q=' + quote(
            keyword.encode(
                'utf-8')) + '&biw=1536&bih=674&tbm=isch&sxsrf=ACYBGNSXXpS6YmAKUiLKKBs6xWb4uUY5gA:1581168823770&source=lnms&sa=X&ved=0ahUKEwioj8jwiMLnAhW9AhAIHbXTBMMQ_AUI3QUoAQ'
        raw_html = _download_page(url)

        end_object = -1;
        google_image_seen = False;
        count = 0
        while count < limit:
            while (True):
                try:
                    new_line = raw_html.find('"https://', end_object + 1)
                    end_object = raw_html.find('"', new_line + 1)

                    buffor = raw_html.find('\\', new_line + 1, end_object)
                    if buffor != -1:
                        object_raw = (raw_html[new_line + 1:buffor])
                    else:
                        object_raw = (raw_html[new_line + 1:end_object])

                    if any(extension in object_raw for extension in {'.jpg', '.png', '.ico', '.gif', '.jpeg'}):
                        break

                except Exception as e:
                    break
            try:
                r = requests.get(object_raw, allow_redirects=True, timeout=1)
                if ('html' not in str(r.content)):
                    file_extension = '.jpg'  # Assume all images are jpg
                    if file_extension == '.png' and not google_image_seen:
                        google_image_seen = True
                        raise ValueError()
                    file_name = f"{keyword}_{count + 1}{file_extension}"
                    save_path = os.path.join(drive_directory, file_name)

                    # Resize the image
                    img = cv2.imdecode(np.asarray(bytearray(r.content)), cv2.IMREAD_COLOR)
                    resized_img = cv2.resize(img, (resize_width, resize_height))

                    # Save resized image to Google Drive
                    cv2.imwrite(save_path, resized_img)
                    bar.update(bar.currval + 1)
                    count += 1
                #else:
                    #count -= 1
            except Exception as e:
                pass
                #count -= 1

    bar.finish()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
real_domain_list = ["dogs", "tiger","lion", "elephant","cow","insects","butterfly","giraffe","deer","birds","flowers","rabbit","peacock","ox","fish","crocodile","Men","Women","children"]

In [ ]:
fake_domain_list = []
for domain in real_domain_list:
    fake_domain_list.append("ai generated "+domain)
print(len(fake_domain_list))

19


In [ ]:
for domain in fake_domain_list:
    print("Domian : "+domain)
    download_and_resize(keywords=domain, limit=1000, directory='Project/AI_Image_classifier/Datasets/Fake', resize_width=300, resize_height=300)

[                                                                        ]   0%

Domian : ai generated dogs


[========================================================================] 100%
[                                                                        ]   0%

Domian : ai generated tiger


[========================================================================] 100%
[                                                                        ]   0%

...

[========================================================================] 100%
[                                                                        ]   0%

Domian : ai generated Women


[========================================================================] 100%
[                                                                        ]   0%

Domian : ai generated children


[===================================================================     ]  93%

In [ ]:
for domain in real_domain_list:
    print("Domain : "+domain)
    download_and_resize(keywords=domain, limit=1000, directory='Project/AI_Image_classifier/Datasets/Real', resize_width=300, resize_height=300)


[                                                                        ]   0%

Domain : dogs


[========================================================================] 100%
[                                                                        ]   0%

Domain : tiger


[========================================================================] 100%
[                                                                        ]   0%

...

[========================================================================] 100%
[                                                                        ]   0%

Domain : Women


[========================================================================] 100%
[                                                                        ]   0%

Domain : children


[========================================================================] 100%


In [ ]:
real_domain_list_2 = ["horse","kangaroo","giraffe","camel","leopard","penguins","polar bear","panda","bear","rabbit","goat","duck","Chicken","parrot","pig","turtle","rat","monkey","snake","zebra","donkey","baby","ostriches","frog","deer","buffalo","fox","swan","sheep","Squirrel"]

In [ ]:
print(len(real_domain_list_2))

30


In [ ]:
for domain in real_domain_list_2:
    print("Domain : "+domain)
    download_and_resize(keywords=domain, limit=1000, directory='Project/AI_Image_classifier/Datasets/Real', resize_width=300, resize_height=300)


[                                                                        ]   0%

Domain : horse


[========================================================================] 100%
[                                                                        ]   0%

Domain : kangaroo


[========================================================================] 100%
[                                                                        ]   0%

...

[========================================================================] 100%
[                                                                        ]   0%

Domain : sheep


[========================================================================] 100%
[                                                                        ]   0%

Domain : Squirrel


[========================================================================] 100%


In [ ]:
fake_domain_list_2 = []
for domain in real_domain_list_2:
    fake_domain_list_2.append("ai generated "+domain)
print(len(fake_domain_list_2))

30


In [ ]:
for domain in fake_domain_list_2:
    print("Domian : "+domain)
    download_and_resize(keywords=domain, limit=1000, directory='Project/AI_Image_classifier/Datasets/Fake', resize_width=300, resize_height=300)

[                                                                        ]   0%

Domian : ai generated horse


[========================================================================] 100%
[                                                                        ]   0%

Domian : ai generated kangaroo


[========================================================================] 100%
[                                                                        ]   0%

...

[========================================================================] 100%
[                                                                        ]   0%

Domian : ai generated sheep


[========================================================================] 100%
[                                                                        ]   0%

Domian : ai generated Squirrel


[========================================================================] 100%
